In [ ]:
import os
import re
import json
from tqdm.notebook import tqdm
from matplotlib import dates
import matplotlib.pyplot as plt
# Choose Style
# 'classic', 'ggplot', 'seaborn', 'Solarize_Light2', 'bmh', 'fivethirtyeight', 'dark_background', 'grayscale',  
plt.style.use('default')

from utils import *
from pprint import pprint

## Find files

In [ ]:
md_files = [
    # '/home/wmnlab/D/database/2023-08-29/2023-08-29.md', 
    # '/home/wmnlab/D/database/2023-09-12-1/2023-09-12-1.md',
    # '/home/wmnlab/D/database/2023-09-22/2023-09-22.md',
    # '/home/wmnlab/D/database/2023-10-24/2023-10-24.md',
    # '/home/wmnlab/D/database/2023-11-21/2023-11-21.md',
    # '/home/wmnlab/D/database/2023-12-26/2023-12-26.md',
    # '/home/wmnlab/D/database/2024-03-10/2024-03-10.md',
    # '/home/wmnlab/D/database/2024-03-16/2024-03-16.md',
    # '/home/wmnlab/E/database/2024-04-18/2024-04-18.md',
    # '/home/wmnlab/E/database/2024-05-08/2024-05-08.md',
    '/home/wmnlab/E/database/2024-05-20/2024-05-20.md'
    ]

EXPs = get_EXPs(md_files)
pprint(EXPs)

## Q1. Average $t_{cmd}$ to RRC reconnection

In [ ]:
diffs = []

for exp in tqdm(EXPs):
    if exp.type not in ['Modem_Action_Test_v6_A']:
        continue
    dev_dir_list = find_device_under_exp(exp.path)
    [dev1, dev2] = list(exp.settings.keys())
    dev_dir_list1 = [d for d in dev_dir_list if dev1 in d]
    dev_dir_list2 = [d for d in dev_dir_list if dev2 in d]
    dev_dir_path1 = dev_dir_list1[0]
    dev_dir_path2 = dev_dir_list2[0]
    
    date_dir = '/'.join(exp.path.split('/')[:-1])
    t_cmd_dir = os.path.join(date_dir, 'others/command_Time')
    t_cmd_files = sorted([os.path.join(date_dir,  'others/command_Time', x)  for x in os.listdir(t_cmd_dir)])
    with open(os.path.join(date_dir, 'sync/time_sync_lpt3.json'),'r') as f:
        deltas = json.load(f)

    trace_dir_list1 = find_trace_under_device(dev_dir_path1)
    trace_dir_list2 = find_trace_under_device(dev_dir_path2)
    for t_num, (t1, t2, cmd_file) in tqdm(enumerate(zip(trace_dir_list1, trace_dir_list2, t_cmd_files)), leave=False):
        rrc1, rrc2 = return_rrc(t1), return_rrc(t2)
        HOs1, HOs2 = parse_mi_ho(rrc1), parse_mi_ho(rrc2)
        t_cmd_df = pd.read_csv(cmd_file)
        delta = list(deltas.values())[t_num]
        j, k = 0, 0
        # pprint(HOs1['Action'])
        # pprint(HOs2['Action'])
        # pprint(delta)
        for i in range(len(t_cmd_df)):
            = t_cmd_df['R1'].iloc[i]
            r2 = t_cmd_df['R2'].iloc[i]
            if == 'action':
                ts = dt.datetime.strptime(t_cmd_df['Timestamp'].iloc[i], '%Y-%m-%d %H:%M:%S.%f')
                ts += dt.timedelta(seconds=delta)
                diff = (HOs1['Action'][j].start - ts).total_seconds()
                print('r1', ts, HOs1['Action'][j].start, diff)
                j+=1
                diffs.append(diff)
            elif r2 == 'action':
                ts = dt.datetime.strptime(t_cmd_df['Timestamp'].iloc[i], '%Y-%m-%d %H:%M:%S.%f')
                ts += dt.timedelta(seconds=delta)
                diff = (HOs2['Action'][k].start - ts).total_seconds()
                print('r2', ts, HOs2['Action'][k].start, diff)
                k+=1
                diffs.append(diff)

In [ ]:
diffs = [d for d in diffs if d < 6]
np.average(diffs), max(diffs), min(diffs)

## Q2. Max Model Action Frequency 

pass

## Q3. Why select the bad choice and what is the probability

In [ ]:
action_times = []
action_exe_times = []
too_late = 0
bad_choice = 0
for exp in tqdm(EXPs):
    if exp.type not in ['Modem_Action_Test_v6_A']:
        continue
    dev_dir_list = find_device_under_exp(exp.path)
    [dev1, dev2] = list(exp.settings.keys())
    dev_dir_list1 = [d for d in dev_dir_list if dev1 in d]
    dev_dir_list2 = [d for d in dev_dir_list if dev2 in d]
    dev_dir_path1 = dev_dir_list1[0]
    dev_dir_path2 = dev_dir_list2[0]
    
    date_dir = '/'.join(exp.path.split('/')[:-1])
    t_cmd_dir = os.path.join(date_dir, 'others/command_Time')
    t_cmd_files = sorted([os.path.join(date_dir,  'others/command_Time', x)  for x in os.listdir(t_cmd_dir)])
    with open(os.path.join(date_dir, 'sync/time_sync_lpt3.json'),'r') as f:
        deltas = json.load(f)

    trace_dir_list1 = find_trace_under_device(dev_dir_path1)
    trace_dir_list2 = find_trace_under_device(dev_dir_path2)
    for t_num, (t1, t2, cmd_file) in tqdm(enumerate(zip(trace_dir_list1, trace_dir_list2, t_cmd_files)), leave=False):
        rrc1, rrc2 = return_rrc(t1), return_rrc(t2)
        HOs1, HOs2 = parse_mi_ho(rrc1), parse_mi_ho(rrc2)
        t_cmd_df = pd.read_csv(cmd_file)
        delta = list(deltas.values())[t_num]

        print(f'Actions {t_num+1}')
        pprint(HOs1['Conn_Req'])
        pprint(HOs2['Conn_Req'])
        
        print('RLF')
        pprint(HOs1['RLF_II']+HOs1['RLF_III'])
        pprint(HOs2['RLF_II']+HOs2['RLF_III'])
        print('Start...')
        j, k = 0, 0
        pre_action1, pre_action2 = None, None

        for i in range(len(t_cmd_df)):
            '= t_cmd_df['R1'].iloc[i]
            r2 = t_cmd_df['R2'].iloc[i]
            
            if == 'action':
                action_time = HOs1['Conn_Req'][j].start
                action_exe_times.append( (HOs1['Conn_Req'][j].end - HOs1['Conn_Req'][j].start).total_seconds() )
                j+=1
                for enb_ho in HOs1['LTE_HO'] + HOs1['MN_HO'] + HOs1['MN_HO_to_eNB']:
                    enb_ho_time = enb_ho.start 
                    if 0 < (action_time - enb_ho_time).total_seconds() < 2: # case too late
                        print('too late1')
                        too_late+=1

                for rlf in HOs1['RLF_II']+HOs1['RLF_III']:
                    rlf_time = rlf.start 
                    if 0 < (action_time - rlf_time).total_seconds() < 3: # case too late
                        print('too late2')
                        too_late+=1
                    elif 0 < (rlf_time - action_time).total_seconds() < 6: # case bad choice
                        print('bad choice1', action_time)
                        bad_choice+=1
                
                if pre_action1 is not None and 0 < (action_time - pre_action1).total_seconds() < 8:
                    print('bad choice2', action_time)
                    bad_choice+=1
                pre_action1 = action_time
                action_times.append(action_time)

            elif r2 == 'action':
                action_time = HOs2['Conn_Req'][k].start
                action_exe_times.append( (HOs2['Conn_Req'][k].end - HOs2['Conn_Req'][k].start).total_seconds() )
                k+=1
                for enb_ho in HOs2['LTE_HO'] + HOs2['MN_HO'] + HOs2['MN_HO_to_eNB']:
                    enb_ho_time = enb_ho.start 
                    if 0 < (action_time - enb_ho_time).total_seconds() < 1: # case too late
                        print('too late1')
                        too_late+=1

                for rlf in HOs2['RLF_II']+HOs2['RLF_III']:
                    rlf_time = rlf.start
                    if 0 < (action_time - rlf_time).total_seconds() < 3:
                        print('too late2')
                        too_late+=1
                    elif 0 < (rlf_time - action_time).total_seconds() < 6: # case bad choice
                        print('bad choice1', action_time)
                        bad_choice+=1
                
                if pre_action2 is not None and 0 < (action_time - pre_action2).total_seconds() < 8:
                    print('bad choice2', action_time)
                    bad_choice+=1
                pre_action2 = action_time
                action_times.append(action_time)

        print('========================')

In [ ]:
len(action_times), too_late , bad_choice, np.average(action_exe_times)

## Q4. Performance difference

In [ ]:
keys = ['DL Loss','DL Excl','UL Loss','UL Excl']
CG5 = {k:[] for k in keys}
v6 = {k:[] for k in keys}

def exp_append(exp, metrics):
    for k, v in zip(keys, metrics):
        exp[k].append(v)

for exp in tqdm(EXPs):
    if exp.type not in ['Modem_Action_Test_v6_A', 'Control_Group5']:
        continue
    dev_dir_list = find_device_under_exp(exp.path)
    [dev1, dev2] = list(exp.settings.keys())
    dev_dir_list1 = [d for d in dev_dir_list if dev1 in d]
    dev_dir_list2 = [d for d in dev_dir_list if dev2 in d]
    dev_dir_path1 = dev_dir_list1[0]
    dev_dir_path2 = dev_dir_list2[0]
    
    date_dir = '/'.join(exp.path.split('/')[:-1])
    t_cmd_dir = os.path.join(date_dir, 'others/command_Time')
    t_cmd_files = sorted([os.path.join(date_dir,  'others/command_Time', x)  for x in os.listdir(t_cmd_dir)])
    with open(os.path.join(date_dir, 'sync/time_sync_lpt3.json'),'r') as f:
        deltas = json.load(f)

    trace_dir_list1 = find_trace_under_device(dev_dir_path1)
    trace_dir_list2 = find_trace_under_device(dev_dir_path2)
    for t_num, (t1, t2, cmd_file) in tqdm(enumerate(zip(trace_dir_list1, trace_dir_list2, t_cmd_files)), leave=False):
        rrc1, rrc2 = return_rrc(t1), return_rrc(t2)
        DL1, DL2 = return_DL(t1), return_DL(t2)
        UL1, UL2 = return_UL(t1), return_UL(t2)

        # dl_loss, dl_excl = count_loss_excl_rate_dual(DL1, DL2)
        # ul_loss, ul_excl = count_loss_excl_rate_dual(UL1, UL2)
        # metrics = [dl_loss*100, dl_excl*100, ul_loss*100, ul_excl*100]
        
        dl_loss_pkg_duals, dl_excl_pkg_duals = loss_excl_cause_dual(DL1, DL2, rrc1, rrc2)
        ul_loss_pkg_duals, ul_excl_pkg_duals = loss_excl_cause_dual(UL1, UL2, rrc1, rrc2)
        
        if exp.type == 'Control_Group5':
            exp_append(CG5, [dl_loss_pkg_duals, dl_excl_pkg_duals, ul_loss_pkg_duals, ul_excl_pkg_duals] )
        elif exp.type == 'Modem_Action_Test_v6_A':
            exp_append(v6, [dl_loss_pkg_duals, dl_excl_pkg_duals, ul_loss_pkg_duals, ul_excl_pkg_duals] )
        

In [ ]:
cause = {}
total_pkg = 0

custom_order = {'Conn_Req': 1, 
                'RLF_III': 2, 
                'RLF_II':3, 
                'SCG_RLF':4, 
                'MN_HO':5,
                'MN_HO_to_eNB':6,
                'SN_HO':7,
                'LTE_HO':8,
                'SN_setup':9,
                'SN_Rel':10}

for L in CG5['DL Excl']:
    for dual_pkg in L:
        total_pkg+=1

        cause1 = [c.split(' ')[-1] for c in dual_pkg.cause1]
        trans1 = dual_pkg.trans1
        cause2 = [c.split(' ')[-1] for c in dual_pkg.cause2]
        trans2 = dual_pkg.trans2
        if len(cause1)==0 and len(cause1)==0:
            continue
        elif len(cause1)==0:
            zipped_lists2 = zip(cause2, trans2)
            sorted_zipped_lists2 = sorted(zipped_lists2, key=lambda x: custom_order[x[0]])  
            cause2, trans2 = zip(*sorted_zipped_lists2)
            for c2, trans2 in zip(cause2, trans2):
                k = f'stable+{c2}'
                if k in cause.keys(): cause[k]+=1
                else: cause[k]=1
                break
            continue
        elif len(cause2)==0:
            zipped_lists1 = zip(cause1, trans1)
            sorted_zipped_lists1 = sorted(zipped_lists1, key=lambda x: custom_order[x[0]])  
            cause1, trans1 = zip(*sorted_zipped_lists1)
            for c1, trans1 in zip(cause1, trans1):
                k = f'{c1}+stable'
                if k in cause.keys(): cause[k]+=1
                else: cause[k]=1
                break
            continue
        zipped_lists1 = zip(cause1, trans1)
        sorted_zipped_lists1 = sorted(zipped_lists1, key=lambda x: custom_order[x[0]])  
        cause1, trans1 = zip(*sorted_zipped_lists1)
        zipped_lists2 = zip(cause2, trans2)
        sorted_zipped_lists2 = sorted(zipped_lists2, key=lambda x: custom_order[x[0]])  
        cause2, trans2 = zip(*sorted_zipped_lists2)
        for c1, trans1 in zip(cause1,trans1):
            pci1 = get_pci_from_HO_type(trans1, c1)
            for c2, trans2 in zip(cause2, trans2):
                pci2 = get_pci_from_HO_type(trans2, c2)
                k = f'{c1}+{c2}'
                if pci1==pci2:
                    k+=' *'
                if k in cause.keys(): cause[k]+=1
                else: cause[k]=1
                break
            break
            
sorted_dict_by_key = dict(sorted(cause.items(), key=lambda item: -item[1]))


In [ ]:
# sorted_dict_by_key, total_pkg

In [ ]:
CG5_loss = {'RLF_II+RLF_II *': 830} # Total 830
# Total 758
CG6_Excl = {'RLF_II+LTE_HO': 471,
            'MN_HO+LTE_HO': 123,
            'SN_HO+stable': 100,
            'SN_HO+LTE_HO': 17,
            'RLF_II+RLF_II *': 17,
            'MN_HO+LTE_HO *': 12,
            'RLF_II+LTE_HO *': 7,
            'SN_HO+LTE_HO *': 6,
            'MN_HO_to_eNB+LTE_HO *': 5}

v6_loss = {'RLF_II+Action *': 790} # Total 790
v6_Excl = {'RLF_II+Action *': 358, # Total 774
           'Action+LTE_HO': 143,
           'MN_HO+Action': 79,
           'SN_HO+LTE_HO *': 57,
           'Action+LTE_HO *': 53,
           'Action+stable': 51,
           'MN_HO_to_eNB+Action *': 33}


In [ ]:
val = list(v6_Excl.values())
lab = [a.replace("_", " ") for a in list(v6_Excl.keys())]
val.reverse(), lab.reverse()
total = 774

In [ ]:
# 創建橫條圖
fig, ax = plt.subplots()
plt.barh(lab, val, color='skyblue')
for i, v in enumerate(x):
    plt.text(v, i, f'{v/total*100:.2f}%')
plt.xlabel('Number of Package (* means having same pci)',weight='bold')
plt.title('Handover Event Pairs vs. Downlink Excessive Latency',weight='bold')
# plt.tight_layout()
plt.ylabel('R1 HO+R2 HO', rotation=0, weight='bold')
ax.yaxis.set_label_coords(-0.13, 0.98)
# 顯示圖表
plt.show()

In [ ]:
CG5_loss = {'RLF_II+RLF_II *': 830} # Total 830
v6_loss = {'RLF_II+Action *': 790} # Total 790
lab = ['B1B8+LTE\nRLF_II+RLF_II *', 'DBL\nRLF_II+Action *']
val = list(CG5_loss.values())+list(v6_loss.values())
x = [0.2,0.5]
fig, ax = plt.subplots(figsize=(4, 4))
plt.bar(x, val, width=0.15, align='center')
plt.xticks(x, lab)
plt.ylabel('Number of package',weight='bold')
plt.xlabel('(* means same pci)',weight='bold')
plt.title('DL Loss Casue of two Scheme', weight='bold')
plt.show()

### Visualize

In [ ]:
base_dir = '/home/wmnlab/E/database'
date = '2024-05-20'
exp = 'Modem_Action_Test_v6_A'
trace = '#09'

date_dir = os.path.join(base_dir, date)
d = os.path.join(date_dir, exp)
devs = sorted([a for a in os.listdir(d) if ('qc' in a or 'sm' in a)])
data_dirs = [os.path.join(d, dev, trace, 'data') for dev in devs]
rrc_files = [[os.path.join(data_dir, f) for f in os.listdir(data_dir) if 'rrc' in f][0] for data_dir in data_dirs]
ml1_files = [[os.path.join(data_dir, f) for f in os.listdir(data_dir) if ('ml1' in f and not 'nr_ml1' in f)][0] for data_dir in data_dirs]
nr_ml1_files = [[os.path.join(data_dir, f) for f in os.listdir(data_dir) if 'nr_ml1' in f][0] for data_dir in data_dirs]
dl_loss_excl_files = [os.path.join(data_dir, 'udp_dnlk_loss_latency.csv') for data_dir in data_dirs]
ul_loss_excl_files = [os.path.join(data_dir, 'udp_uplk_loss_latency.csv') for data_dir in data_dirs]


In [ ]:
ordered_HOs_list = [to_ordered_HO(f) for f in rrc_files]
dl_loss_ecxl_pkgs = [loss_excl_cause(dl, rrc) for dl, rrc in zip(dl_loss_excl_files, rrc_files)]
ul_loss_ecxl_pkgs = [loss_excl_cause(ul, rrc) for ul, rrc in zip(ul_loss_excl_files, rrc_files)]

In [ ]:
dl_loss1, dl_excl1 = loss_excl_cause_dual(dl_loss_excl_files[0], dl_loss_excl_files[1], rrc_files[0], rrc_files[1])
dl_loss2, dl_excl2 = loss_excl_cause_dual(dl_loss_excl_files[2], dl_loss_excl_files[3], rrc_files[2], rrc_files[3])
len(dl_loss1), len(dl_excl1), len(dl_loss2), len(dl_excl2)

In [ ]:
start_time = dt.datetime(2024, 5, 20, 16, 44, 30, 000000)
end_time = dt.datetime(2024, 5, 20, 16, 44, 37, 000000)
time_range = (start_time, end_time)

In [ ]:
elements = []
for ordered_HOs in ordered_HOs_list:
    T, Type, Trans, Ev = get_info(ordered_HOs, time_range)
    elements.append( (T, Type, Trans, Ev) )

fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True) 
HO_SCAs = []

labels = ['HO (CG5)', 'HO (CG5)', 'R1 HO (DBL)', 'R2 HO (DBL)']
colors = ['r', 'b'] * 2
for element, label, c in zip(elements, labels, colors):
    (T, Type, Trans, Ev) = element
    i = 0 if 'CG5' in label else 1
    HO_SCA = axes[i].scatter(T, [1] * len(T), marker='*', color=c, s=100, zorder=3, label=label)
    HO_SCAs.append(HO_SCA)

    pos = 1.1 if 'R1' in label else 0.65
    for event, type, trans in zip(T, Type, Trans):                          
        if type in ['LTE HO', 'MN HO']:
            trans = trans.split(' | ')[0]
            text = type + '\n' + get_pci(trans, 'eNB HO')
        elif type in ['RLF II']:
            trans = trans.split(' | ')[0]
            text = type + '\n' + get_pci(trans, 'eNB HO')
        elif type == 'MN HO to eNB':
            trans = trans.split(' | ')[0]
            text = 'MN HO\nto eNB'+ '\n' + get_pci(trans, 'eNB HO')
        elif type in ['SN HO']:
            trans = trans.split(' | ')[1]
            text = type + '\n' + get_pci(trans, 'gNB HO')
        elif type == 'SN setup':
            trans = trans.split(' | ')[1]
            text = type + '\n' + get_pci(trans, 'gNB setup')
        elif type == 'Conn Req':
            text = type + '\n' + get_pci(trans, 'eNB HO')
        elif type == 'SCG RLF':
            text = 'SCG\nRLF'
        elif type == 'SN Rel':
            text = 'SN\nRel' + '\n' + get_pci(trans, 'gNB rel')
        
        axes[i].text(event, pos, text, ha='center', va='bottom', fontsize=8, color=c, zorder=3)

for i in range(2):
    axes[i].axhline(y=1, color='gray', linestyle='--', linewidth=1)
    axes[i].set_yticks([])
    axes[i].set_ylim([0,2])
    axes[i].set_xlim((start_time, end_time))

DL_Losses = []
DL_Excls = []
labels = ['R1 Loss (CG5)', 'R2 Loss (CG5)', 'R1 Loss (DBL)', 'R2 Loss (DBL)']
labels2 = ['R1 Excl (CG5)', 'R2 Excl (CG5)', 'R1 Excl (DBL)', 'R2 Excl (DBL)']
colors = ['lightblue', 'lightpink'] * 2; colors2 = ['lightgreen', 'gold'] * 2
y_range = axes[0].get_ylim()
for (loss, excl), lab, lab2, c, c2 in zip(dl_loss_ecxl_pkgs, labels, labels2, colors, colors2):
    L = [pkg.timestamp for pkg in loss]; L2 = [pkg.timestamp for pkg in excl]
    a,b = (1, y_range[1]) if 'R1' in lab else (y_range[0], 1)
    i = 0 if 'CG5' in lab else 1
    DL_LOSS = axes[i].vlines(x=L, ymin=a, ymax=b, alpha=1, zorder=2, colors=c, label=lab)
    DL_EXCL = axes[i].vlines(x=L2, ymin=a, ymax=b, alpha=1, zorder=1, colors=c2, label=lab2)
    DL_Losses.append(DL_LOSS)
    DL_Excls.append(DL_EXCL)


for i in range(2):
    axes[i].legend(loc='upper right',  bbox_to_anchor=(1.20, 1.1))

myFmt = dates.DateFormatter('%H:%M:%S') # fmt %H:%M:%S.%f
axes[-1].xaxis.set_major_formatter(myFmt)
axes[-1].set_xlabel('Time')
fig.autofmt_xdate()
plt.show()